In [1]:
##### connect to the elasticsearch server
import json
from pprint import pprint
import os
import time

from dotenv import load_dotenv
from elasticsearch import Elasticsearch

load_dotenv()

class Search:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.es = Elasticsearch(
            "http://localhost:9200", # deployed locally, no cloud
            basic_auth=(username, password))  # <-- connection options need to be added here
        client_info = self.es.info()
        print('Connected to Elasticsearch!')
        pprint(client_info.body)
        
    def create_index(self, index_name, metadata_profile):
        self.es.indices.create(index = index_name,  mappings = metadata_profile)

##### print the client info, check if the elasticsearch server is running
my_es = Search("elastic", "genov4")

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'NsOWoTPiSUuzdR39O9X-4w',
 'name': '374a4a1dbfca',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-07-01T22:06:58.515911606Z',
             'build_flavor': 'default',
             'build_hash': '2afe7caceec8a26ff53817e5ed88235e90592a1b',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.10.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.14.2'}}


In [12]:
metadata_profile = {
    "properties": {
        "SampleID": {
            "type": "text"
        },
        "FileName": {
            "type": "text"
        },
        "author": {
            "type": "text"
        },
        "created": {
            "type": "date"
        },
        "pipeline": {
            "type": "keyword"
        }
    }

}
my_es.create_index(index_name = "test", metadata_profile = test_metadata_profile)

/var/folders/8j/g55jqp3s7mj16s_gvcwb4y800000gn/T/ipykernel_88590/3076276374.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.es.indices.create(index='test-index', ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test-index'})